<a href="https://colab.research.google.com/github/acts701/Audio/blob/main/250118_FAISS_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kss
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 52.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 14.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.3/131.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.9/757.9 kB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.4/939.4 kB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.2/72.2 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import json
import kss
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from google.colab import drive
from tqdm import tqdm

# Google Drive 마운트
drive.mount('/content/drive')

# 사용자 설정
originTxt_folder_path = '/content/drive/MyDrive/LLM/Data/voiceToText/clovaPaid'  # STT 파일 경로
output_folder = '/content/drive/MyDrive/LLM/Data/vectorDBChunks'  # JSON 청크 저장 경로
os.makedirs(output_folder, exist_ok=True)

Mounted at /content/drive


In [3]:
# SentenceTransformer 모델 로드
model = SentenceTransformer("jhgan/ko-sbert-multitask")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.78k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
# STT 파일 처리
file_list = sorted(os.listdir(originTxt_folder_path))
for file_name in tqdm(file_list, desc="Processing Files"):
    file_path = os.path.join(originTxt_folder_path, file_name)
    if not os.path.isfile(file_path):
        continue  # 파일이 아닌 경우 스킵
    # 파일 읽기 및 문장 분리
    with open(file_path, 'r', encoding='utf-8') as file:
        contents = file.read()
        sentences = kss.split_sentences(contents)

    # 벡터 데이터 저장 리스트
    vector_chunk = []

    # 문장 벡터 생성 및 저장
    for idx, sentence in enumerate(sentences):
        sentence = sentence.strip()
        if not sentence:  # 빈 문장 무시
            continue
        try:
            embedding = model.encode(sentence).tolist()

            vector_entry = {
                "id": f"{file_name}-{idx}",  # 파일 이름과 문장 번호로 고유 ID 생성
                "text": sentence,
                "vector": embedding
            }
            vector_chunk.append(vector_entry)
        except Exception as e:
            print(f"문장 벡터화 오류: {sentence}\n{e}")

    # 파일 이름 기반 JSON 파일 경로 생성
    json_file_name = f"{os.path.splitext(file_name)[0]}.json"  # 확장자를 .json으로 변경
    chunk_file_path = os.path.join(output_folder, json_file_name)

    # JSON 파일로 저장
    try:
        with open(chunk_file_path, 'w', encoding='utf-8') as f:
            json.dump(vector_chunk, f, ensure_ascii=False, indent=4)
            print(f"json saved {chunk_file_path}")
    except Exception as e:
        print(f"json dump failed: {e}")

    print(f"{file_name}에 대한 벡터 데이터가 저장되었습니다: {chunk_file_path}")

print(f"모든 파일이 처리되고 JSON 파일로 저장되었습니다. 저장 경로: {output_folder}")

Processing Files:   0%|          | 0/3 [00:00<?, ?it/s]WARNING:root:Oh! You have mecab in your environment. Kss will take this as a backend! :D

Processing Files:  33%|███▎      | 1/3 [00:11<00:22, 11.22s/it]

json saved /content/drive/MyDrive/LLM/Data/vectorDBChunks/SA_1_1_9.json
SA_1_1_9.txt에 대한 벡터 데이터가 저장되었습니다: /content/drive/MyDrive/LLM/Data/vectorDBChunks/SA_1_1_9.json


Processing Files:  67%|██████▋   | 2/3 [00:23<00:12, 12.12s/it]

json saved /content/drive/MyDrive/LLM/Data/vectorDBChunks/SA_1_21_31.json
SA_1_21_31.txt에 대한 벡터 데이터가 저장되었습니다: /content/drive/MyDrive/LLM/Data/vectorDBChunks/SA_1_21_31.json


Processing Files: 100%|██████████| 3/3 [00:36<00:00, 12.25s/it]

json saved /content/drive/MyDrive/LLM/Data/vectorDBChunks/SA_2_1_5.json
SA_2_1_5.txt에 대한 벡터 데이터가 저장되었습니다: /content/drive/MyDrive/LLM/Data/vectorDBChunks/SA_2_1_5.json
모든 파일이 처리되고 JSON 파일로 저장되었습니다. 저장 경로: /content/drive/MyDrive/LLM/Data/vectorDBChunks


In [14]:
# FAISS 인덱스 생성 및 벡터 추가
vector_data = []  # 전체 벡터 데이터 저장
metadata = []  # 메타데이터 저장

chunk_files = sorted(os.listdir(output_folder))
for chunk_file in chunk_files:
    if "pinecone" in chunk_file:  # 파일 이름에 "pinecone"이 포함된 경우 건너뜀
        continue
    chunk_path = os.path.join(output_folder, chunk_file)
    with open(chunk_path, 'r', encoding='utf-8') as f:
        vector_chunk = json.load(f)
        for entry in vector_chunk:
            vector_data.append(entry['vector'])
            metadata.append({"id": entry['id'], "text": entry['text']})

# FAISS 인덱스 생성
dimension = len(vector_data[0])
faiss_index = faiss.IndexFlatL2(dimension)
faiss_index.add(np.array(vector_data, dtype='float32'))

print("FAISS 인덱스 생성 및 데이터 추가 완료.")

FAISS 인덱스 생성 및 데이터 추가 완료.


In [39]:
# 검색 테스트
query_text = "성경이 말하는 예언 선지자들의 말씀은"
try:
    query_vector = np.array([model.encode(query_text)], dtype='float32')

    # FAISS 검색 수행
    top_k = 10
    distances, indices = faiss_index.search(query_vector, k=top_k)

    print("검색 결과:")
    for i, idx in enumerate(indices[0]):
        result = metadata[idx]
        print(f"Rank {i+1}:\nID: {result['id']}\nOriginal Text: {result['text']}\nDistance: {distances[0][i]}\n")

except Exception as e:
    print(f"검색 오류: {e}")

검색 결과:
Rank 1:
ID: SA_1_1_9.txt-68
Original Text: 성경이 말하는 예언 선지자들의 말씀은 이제 이사야서부터 시작되는 이사야 예레미야 에스겔 다니엘 그리고 열두 선지자들로 얘기하는이 선지서들의 말씀은 지금 당대에 우리 시대에이 사람들 살고 있는이 시대의 하나님의 뜻을 전달하는 것이며 만약에 너희들이 하나님의 뜻을 받지 않으면은 어떻게 어떻게 될 것이다라는 말을 하고 있는 것입니다뿐만 아니라 하나님은이 시대를 어떻게 어떤 뜻을 갖고 계신지 이야기한 다음에 장차 이들을 향한 어떤 계획을 갖고 계신지를 말씀하고 있는 정도이지 개인의 미래에 대해서 얘기하려고 하는 것이 아니라 사실입니다
Distance: 160.42111206054688

Rank 2:
ID: SA_1_1_9.txt-60
Original Text: 선지자들의 말씀을 우리는 예언이라 이렇게 얘기하고 있는데 오늘 한국에서는 예언의 은사 이렇게 해서 예언이란 말이에요
Distance: 182.5118408203125

Rank 3:
ID: SA_1_1_9.txt-139
Original Text: 이사야는이 시대에 하나님의 이상을 통해서 하나님의 말씀을 깨닫고 이들에게 말씀을 전하고 있는 것입니다
Distance: 258.31170654296875

Rank 4:
ID: SA_1_1_9.txt-66
Original Text: 이런 말을 쓰고 있는데 성경은 예언이란 말을 그런 말로 쓰고 있지 않습니다
Distance: 268.99267578125

Rank 5:
ID: SA_2_1_5.txt-800
Original Text: 종말론을 배우면 우리가 뭘 해야 될지를 나타나는 것이 그래서 여러분 요한계시록이 종말론을 다루는데 희한한 얘기를 하는 것처럼 보이지만 요한계시록은 교회에 딱 뭘 당부하는 말입니까?
Distance: 271.93145751953125

Rank 6:
ID: SA_1_1_9.txt-88
Original Text: 그래서 모든 선지자들은 다 어떤 상황에 처한